# Transformer Preliminaries 


## Framing
> GOAL: Create a notebook with proof-of-concepts/examples for each component of the upcoming transformer/Perceiver architecture. 

Objectives: 
 1. Gain familiarity with the code side of transformers.
 2. Create examples of each unfamiliar components of the project. 
 3. Maximize copy-paste-ability. 


## Diverging 
> Things to prototype.

 - [ ] Patch extraction.
	 - [ ] Video.
	 - [ ] Audio. 
	 - [ ] VTT subtitle files. 
 - [ ] Text tokenization/vectorization. 
 - [ ] Implementation of spacetime Fourier features (positional encoding).
 - [ ] Attention blocks (self attention, cross attention).
	 - Should be a `dmodel` or whatever Tensorflow uses. 
 - [ ] Masking for transformer layers. 
 - [ ] Decoder/encoder cold start (cf. Perceiver paper).
 - [ ] How to query latent space for specific spatiotemporal coordinates?